# Electronic Structure Calculations using Generalized Ising Hamiltonians

Your final task is to calculate the ground state energy of a hydrogen molecule using a Generalized Ising Hamiltonian:

$$ 
H = E_0 + \sum_i h_i\sigma_i + \sum_{ij} J_{ij}\sigma_i\sigma_j
+ \sum_{ijk} K_{ijk}\sigma_i\sigma_j\sigma_k
+ \sum_{ijkl} L_{ijkl}\sigma_i\sigma_j\sigma_k\sigma_l
+ \cdots
$$

where the Hamiltonian parameters ($E_0, h_i, J_{ij}, K_{ijk}, L_{ijkl}$) will be provided to you by the `read_generalized_ising_hamiltonian` function below.

These Ising Hamiltonians were produced using the *Iterative Qubit Coupled Cluster* method (https://arxiv.org/abs/1906.11192).

The $H_2$ Ising Hamiltonian only needs 4 spins to fully describe the ground state, hence our Hamiltonian only needs a 4-point interaction at most.

In [1]:
import numpy as np

from abstract_ising import AbstractIsing
from ising_animator import IsingAnimator

%matplotlib inline

In [2]:
def read_generalized_ising_hamiltonian(path):
    with open(path, "r") as f:
        f.readline()  # discard first line
        compressed_hamiltonian = [
            tuple(line.strip().split())
            for line in f.readlines()
        ]
    
    num_sites = len(compressed_hamiltonian[0][0])
    hamiltonian_terms = [np.zeros((num_sites,)*i) for i in range(num_sites+1)]

    for sites, val in compressed_hamiltonian:
        num_zs = 0
        site_nums = []
        for i, x in enumerate(sites):
            if x == 'z':
                site_nums.append(i)
                num_zs += 1

        hamiltonian_terms[num_zs][tuple(site_nums)] = float(val)

    return hamiltonian_terms

In [10]:
# Get Hamiltonian Parameters
E0, h, J, K, L = read_generalized_ising_hamiltonian("./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=0.65.inp")

First, you must write a class that describes the Generalized Ising Model given the arrays containing the Hamiltonian parameters. You will need to write a function which computes energy of the stored spin configuration (see the previous tasks for inspiration), as well as a function which computes the change in energy resulting from a single-spin-flip (you could of course use the naive approach and simply compute the energy of two spin configurations and subtract one from the other).

In [32]:
# Write a class to describe the generalized Ising Model given its Hamiltonian parameters
# Only need 4 spins
class GeneralizedIsingModel(AbstractIsing):
    def __init__(self, N, E0, h, J, K, L):
        self.N = N
        self.E0 = E0
        self.h = h
        self.J = J
        self.K = K
        self.L = L
        self.num_spins = self.N
        
        # initialize system at infinite temperature
        # i.e. spins are completely random and uncorrelated
        self.spins = 2*(np.random.rand(self.N) < 0.5) - 1
    
    def energy(self,spins=None):
        # hint: np.einsum may be helpful here
        """Returns the energy of the current spin configuration"""
        spins = self.spins if spins is None else spins
        # E0 + h_i + J_ij + K_ijk + L_ijkl
        # Einsum kicks ass!
        total = (self.E0 + np.einsum("i,i",spins,h) + np.einsum("i,j,ij", spins, spins, J) + 
                 np.einsum("i,j,k,ijk", spins, spins, spins, K) + np.einsum("i,j,k,l,ijkl",spins,spins,spins,spins,L))
        return total
        
    
    def energy_diff(self, i):
        """Returns the energy difference resulting from flipping the i'th site"""
        current_energy = self.energy()
        # Flip the spins 
        self.spins[i] *= -1
        # Get new energy
        new_energy = self.energy()
        # Flip spins back
        self.spins[i] *= -1
        
        return (new_energy - current_energy)
        
    
    def rand_site(self):
        """Selects a site in the lattice at random"""
        return (np.random.randint(self.N),)
    


Next you'll run a Monte Carlo simulation for this model at some finite temperature for 1000 steps, printing out the energy of the state every so often

In [42]:
ising = GeneralizedIsingModel(4,E0, h, J, K, L)
# your MC simulation here
# perform 1000 MC steps
print("=== Monte Carlo Steps for 4 spins ===")
for t in range(1000):

    E = ising.mc_step(T=1.0)
    
    # Print out E ever so often...
    if t % 50 == 0:
        print(E)

=== Monte Carlo Steps for 4 spins ===
-0.333193379859456
-0.48557107681859907
-1.1299047752322906
-0.4020519317130593
-0.33319337985945613
0.3973123804093819
-0.40205193171305953
-0.40205193171305953
-0.3331933798594561
-1.1299047752322906
0.8171587078823377
1.144680802606227
-0.40205193171305953
-0.40205193171305953
-0.4855710768185992
-0.33319337985945596
0.5724567781741281
-1.1299047752322906
0.39731238040938205
-1.1299047752322906


Now, apply (one of) the annealing procedure(s) you came up with in the previous task to this problem to find a ground state of the system:

In [44]:
# your annealing code here
def annealing_schedule(steps, start_temp, final_temp):
    N = steps
    T_i = start_temp
    T_f = final_temp
    for t in range(N):
        # Annealing
        T = T_i * ((T_f/T_i) ** (t/N))
        E = ising.mc_step(T=T)

        if t % 50 == 0:
            print(E)
            
    return E

# Steps in MC
N = 1000

# Starting temperature
T_i = 1000

# Final Temperature
T_f = 0.01

print("\n=== Annealing Energy Calculation for 4 spins ===")
annealed_energy = annealing_schedule(N,T_i,T_f)


=== Annealing Energy Calculation for 4 spins ===
-0.485571076818599
-0.19222515402425983
-0.19222515402425988
0.8171587078823377
-0.4020519317130593
-0.33319337985945596
0.5724567781741281
-0.4855710768185991
0.5724567781741281
0.5724567781741281
-0.485571076818599
0.7256454984385436
-0.40205193171305936
0.5724567781741281
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906


Finally, iterate over the entire spin configuration space (this is tractable since we only have 4 spins) to find the exact ground state energy. Compare this energy to the one you found above using your annealer.

In [45]:
# exact ground state calculation
# This is getting ground state exactly  
# N, E0, h, J, K, L
#ising = GeneralizedIsingModel(4,E0, h, J, K, L)
dim = np.arange(2 ** ising.num_spins)
space = ((dim[:, None] & (1 << np.arange(ising.num_spins))) > 0)
space = 2*space.astype(int) - 1

exact_energy = 0
for row in space:
    exact_energy = ising.energy().min()
    
#ising.energy(space).min()
print("=== Exact Energy Calculation for 4 spins ===")
print(exact_energy.min())
#print(ising.energy(space).min())

=== Exact Energy Calculation for 4 spins ===
-1.1299047752322906


Now, clean up your code a little, and write a for-loop that iterates over all the different values of the Hydrogen seperation distance $R$ available in the `hamiltonians` directory. 

For each $R$ you must:
- Read in the associated Ising Hamiltonian
- Perform an annealed Monte Carlo simulation to find a candidate ground state energy
- Compute the exact ground state energy
- Compare the two results

In [50]:
import os

N = 1000

# Starting temperature
T_i = 1000

# Final Temperature
T_f = 0.01

def exact_energy(ising):
    dim = np.arange(2 ** ising.num_spins)
    space = ((dim[:, None] & (1 << np.arange(ising.num_spins))) > 0)
    space = 2*space.astype(int) - 1

    exact = 0
    for row in space:
        exact = ising.energy().min()
    
    return exact
    
# Open all files in hamiltonians directory
for root, dirs, files in os.walk("./hamiltonians"):
    for file in files:
        if file.endswith(".inp"):  #only get .inp hamiltonian files
            ham_file = os.path.join(root, file)
            print("\n***** FILE: " + os.path.splitext(ham_file)[0] + " *****")
            E0, h, J, K, L = read_generalized_ising_hamiltonian(ham_file)
            # Anneal
            ising = GeneralizedIsingModel(4,E0, h, J, K, L)
            print("\n=== Annealing Energy Calculation for 4 spins ===")
            annealed_energy = annealing_schedule(N,T_i,T_f)
            print(annealed_energy)
            print("\n=== Exact energy Calucluation 4 spins ===")
            exact = exact_energy(ising)
            print(exact)
            print("\n=== Difference Between Exact and Annealed ===")
            print(exact - annealed_energy)
            
                


***** FILE: ./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=3.05 *****

=== Annealing Energy Calculation for 4 spins ===
-0.9322838369715035
-0.9322838369715035
-0.5536235300754706
-0.5389082449229711
0.0045045760747658425
-0.5482652812703132
-0.9335506069059255
-0.548265281270313
-0.5536235300754706
-0.5536235300754706
-0.5414871387155734
-0.9335506069059255
-0.5389082449229711
0.0045045760747658425
-0.9329778727641554
-0.9322838369715035
-0.5536235300754706
-0.9329778727641554
-0.9329778727641554
-0.9329778727641554
-0.9329778727641554

=== Exact energy Calucluation 4 spins ===
-0.9329778727641554

=== Difference Between Exact and Annealed ===
0.0

***** FILE: ./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=1.1 *****

=== Annealing Energy Calculation for 4 spins ===
-0.15503966063212382
-0.0683012289153341
-0.06830122891533408
-0.5646228370449184
-0.7929596689353228
0.3979927305983758
-0.15503966063212382
0.3979927305983757
-0.6714460419903794
-0.5646228370449184
-1.0791929635915072
-0.61

-0.9001129139196995
-0.9001129139196995
-0.9245373170007651
-0.9486411206967023
-0.9486411206967023

=== Exact energy Calucluation 4 spins ===
-0.9486411206967023

=== Difference Between Exact and Annealed ===
0.0

***** FILE: ./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=0.65 *****

=== Annealing Energy Calculation for 4 spins ===
1.144680802606227
-0.4020519317130593
0.7256454984385436
-1.1299047752322906
-0.25924181243352884
0.397312380409382
-0.48557107681859907
-0.3331933798594561
-0.19222515402425983
-0.33319337985945596
1.144680802606227
-1.1299047752322906
-0.40205193171305953
-1.1299047752322906
-0.485571076818599
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906

=== Exact energy Calucluation 4 spins ===
-1.1299047752322906

=== Difference Between Exact and Annealed ===
0.0

***** FILE: ./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=3.2 *****

=== Annealing Energy Calculation for 4 spins ===
-0.933379979185252